In [1]:
%pip show semantic-kernel

Name: semantic-kernelNote: you may need to restart the kernel to use updated packages.

Version: 0.3.10.dev0
Summary: 
Home-page: 
Author: Microsoft
Author-email: SK-Support@microsoft.com
License: 
Location: c:\git\dalleflask\.venv\lib\site-packages
Requires: aiofiles, numpy, openai, openapi_core, prance, pydantic, python-dotenv, regex
Required-by: 


In [2]:
# https://github.com/microsoft/semantic-kernel/blob/main/python/README.md


import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureChatCompletion

import dotenv
import os

dotenv.load_dotenv()

True

In [3]:
# create a python logger
import logging
logger = logging.getLogger(__name__)



In [4]:
# get the above from os.environ.get and use provided values as defaults
chatgpt_api_key = os.environ.get('chatgpt_api_key', '121212121212')
chatgpt_region = os.environ.get('chatgpt_region', 'southcentralus')
chatgpt_endpoint = os.environ.get('chatgpt_endpoint', 'https://magopenai.openai.azure.com/')
chatgpt_api_version = os.environ.get('chatgpt_api_version', '2021-03-15-preview')



# get above env variables using os.environ.get and use provided values as defaults
gpt4_endpoint = os.environ.get('gpt4_endpoint', 'https://lnc-eastus-openai.openai.azure.com/')
gpt4_key = os.environ.get('gpt4_key', '121221')
gpt4_region = os.environ.get('gpt4_region', 'eastus')
gpt4_modelid_4k = os.environ.get('gpt4_modelid_4k', 'gpt-4-0314')
gpt4_modelid_32k = os.environ.get('gpt4_modelid_32k', 'gpt-4-32k-0314')
gpt4_api_version = os.environ.get('gpt4_api_version', '2023-03-15-preview')

In [5]:
# sk.azure_openai_settings_from_dot_env()

kernel = sk.Kernel(log=logger)
kernel.add_text_completion_service(
    "dv",
    AzureTextCompletion(
    deployment_name='davinci',
    endpoint=chatgpt_endpoint,
    api_key=chatgpt_api_key,
    api_version=chatgpt_api_version,
    logger=logger,
    ad_auth=False,
    )
)


kernel.add_chat_service(
    "gpt4",
    AzureChatCompletion(
    deployment_name=gpt4_modelid_4k,
    endpoint=gpt4_endpoint,
    api_key=gpt4_key,
    api_version=gpt4_api_version,
    logger=logger,
    ad_auth=False,
    )
)






In [6]:
prompt = kernel.create_semantic_function("""
1) A robot may not injure a human being or, through inaction,
allow a human being to come to harm.

2) A robot must obey orders given it by human beings except where
such orders would conflict with the First Law.

3) A robot must protect its own existence as long as such protection
does not conflict with the First or Second Law.

Give me the TLDR.""")

In [7]:
# kernel.set_default_text_completion_service('dv')

print(prompt())

# kernel.set_default_text_completion_service('gpt4')

# print(prompt())



Robots must not harm humans, obey orders from humans, and protect themselves as long as it does not conflict with the first two laws.


In [8]:
pp = kernel.create_semantic_function("""1) A robot may not injure a human being or, through inaction,
allow a human being to come to harm.

2) A robot must obey orders given it by human beings except where
such orders would conflict with the First Law.

3) A robot must protect its own existence as long as such protection
does not conflict with the First or Second Law.

Give me the TLDR.""")

In [9]:
print(pp())



Robots must not harm humans, obey orders from humans, and protect themselves as long as it does not conflict with the first two laws.


In [10]:
kernel._default_text_completion_service

'dv'

In [11]:
kernel.set_default_text_completion_service('gpt4')

# Create a reusable function with one input parameter
summarize = kernel.create_semantic_function("{{$input}}\n\nOne line TLDR with the fewest words.")

# Summarize the laws of thermodynamics
print(summarize("""
1st Law of Thermodynamics - Energy cannot be created or destroyed.
2nd Law of Thermodynamics - For a spontaneous process, the entropy of the universe increases.
3rd Law of Thermodynamics - A perfect crystal at zero Kelvin has zero entropy."""))

# Summarize the laws of motion
print(summarize("""
1. An object at rest remains at rest, and an object in motion remains in motion at constant speed and in a straight line unless acted on by an unbalanced force.
2. The acceleration of an object depends on the mass of the object and the amount of force applied.
3. Whenever one object exerts a force on another object, the second object exerts an equal and opposite on the first."""))

# Summarize the law of universal gravitation
print(summarize("""
Every point mass attracts every single other point mass by a force acting along the line intersecting both points.
The force is proportional to the product of the two masses and inversely proportional to the square of the distance between them."""))

# Output:
# > Energy conserved, entropy increases, zero entropy at 0K.
# > Objects move in response to forces.
# > Gravitational force between two point masses is inversely proportional to the square of the distance between them.

1st Law: Energy conserved; 2nd Law: Entropy increases; 3rd Law: Perfect crystal, zero Kelvin, zero entropy.
Objects' motion depends on forces; acceleration relies on mass and force; forces occur in equal, opposite pairs.
Masses attract inversely to distance squared.


In [ ]:
# from semantic_kernel
kernel.all_text_completion_services()

In [ ]:
kernel.all_chat_services()

In [ ]:
kernel.all_text_embedding_generation_services()

In [12]:
messages=[
    { "role": "system",    "content": "You are a helpful assistant."},
    { "role": "user",      "content": "Who won the world series in 2020?"},
    { "role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    { "role": "user",      "content": "Where was it played?"}
]

In [13]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'Who won the world series in 2020?'},
 {'role': 'assistant',
  'content': 'The Los Angeles Dodgers won the World Series in 2020.'},
 {'role': 'user', 'content': 'Where was it played?'}]

In [16]:
chatGPT = kernel._default_chat_service

In [17]:
chatGPT

'gpt4'

In [18]:
type(chatGPT)

str

In [19]:
# from semantic_kernel.kernel import ChatCompletionClientBase

In [21]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [22]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)


In [23]:
context = kernel.create_new_context()
context["history"] = ""

In [37]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)
print(type(bot_answer))

Sure, I'd be happy to help! What genre or type of book are you interested in?
<class 'semantic_kernel.orchestration.sk_context.SKContext'>


In [43]:
context["user_input"] = "I like science fiction"
print(await chat_function.invoke_async(context=context))

That's great! Science fiction is a fascinating genre that explores imaginative and futuristic concepts. Do you have any favorite science fiction books, movies, or TV shows?


In [44]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: I like science fiction
ChatBot: That's great! Science fiction is a fascinating genre that explores imaginative and futuristic concepts. Do you have any favorite science fiction books, movies, or TV shows?



In [47]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [48]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Sure! Ancient Greece was a highly influential civilization known for its contributions to various fields such as philosophy, science, arts, and politics. Some key figures from Ancient Greece include philosophers like Socrates, Plato, and Aristotle, who have greatly impacted Western thought. You might be interested in reading about the city-states of Athens and Sparta, which had very different political systems and ways of life. Additionally, you could explore Greek mythology, which is full of fascinating stories and characters. To learn more, you can read books, watch documentaries, or take online courses on Ancient Greece and its culture.


In [50]:
from pprint import pprint
pprint(context["history"])

('\n'
 'User: I like science fiction\n'
 "ChatBot: That's great! Science fiction is a fascinating genre that explores "
 'imaginative and futuristic concepts. Do you have any favorite science '
 'fiction books, movies, or TV shows?\n'
 '\n'
 "User: I love history and philosophy, I'd like to learn something new about "
 'Greece, any suggestion?\n'
 'ChatBot: Sure! Ancient Greece was a highly influential civilization known '
 'for its contributions to various fields such as philosophy, science, arts, '
 'and politics. Some key figures from Ancient Greece include philosophers like '
 'Socrates, Plato, and Aristotle, who have greatly impacted Western thought. '
 'You might be interested in reading about the city-states of Athens and '
 'Sparta, which had very different political systems and ways of life. '
 'Additionally, you could explore Greek mythology, which is full of '
 'fascinating stories and characters. To learn more, you can read books, watch '
 'documentaries, or take online cou

In [51]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Ancient Greece is known for its rich history, culture, and contributions to various fields. It is often considered the birthplace of Western civilization. Some key aspects of Ancient Greece include:

1. Philosophy: Greek philosophers like Socrates, Plato, and Aristotle laid the foundations for Western philosophy. They explored topics such as ethics, metaphysics, politics, and epistemology.

2. City-states: Ancient Greece was made up of independent city-states, such as Athens and Sparta, each with its own government, culture, and way of life. Athens was known for its democracy and focus on arts and education, while Sparta was a militaristic society with a strict social hierarchy.

3. Mythology: Greek mythology is a collection of stories and legends about gods, heroes, and mythical creatures. These stories often served as explanations for natural phenomena or as moral lessons for the people. Some well-known Greek myths include the

In [52]:
from pprint import pprint
pprint(context["history"])

('\n'
 'User: I like science fiction\n'
 "ChatBot: That's great! Science fiction is a fascinating genre that explores "
 'imaginative and futuristic concepts. Do you have any favorite science '
 'fiction books, movies, or TV shows?\n'
 '\n'
 "User: I love history and philosophy, I'd like to learn something new about "
 'Greece, any suggestion?\n'
 'ChatBot: Sure! Ancient Greece was a highly influential civilization known '
 'for its contributions to various fields such as philosophy, science, arts, '
 'and politics. Some key figures from Ancient Greece include philosophers like '
 'Socrates, Plato, and Aristotle, who have greatly impacted Western thought. '
 'You might be interested in reading about the city-states of Athens and '
 'Sparta, which had very different political systems and ways of life. '
 'Additionally, you could explore Greek mythology, which is full of '
 'fascinating stories and characters. To learn more, you can read books, watch '
 'documentaries, or take online cou